<a href="https://colab.research.google.com/github/barry80417/titanic-analysis/blob/main/titanic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [264]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df_train = pd.read_csv("https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/agconti/kaggle-titanic/master/data/test.csv")

用github內的資料直接讀進來

In [ ]:
#@title data information
df_train.head()
df_train.info()

 Link: https://www.kaggle.com/competitions/titanic/data

Survived: Survived (1) or died (0)

Pclass: Passenger’s class

Name: Passenger’s name

Sex: Passenger’s sex

Age: Passenger’s age

SibSp: define brother,sister,husband,wife...

Parch: define mother,father,daughter,son...

Ticket: ticket number

Fare:ticket fare

Cabin:cabin number

Embarked:port of embarkation (C=Cherbourg,Q=Queenstown,S=Southampton)

In [ ]:
#@title train data relation
#看訓練資料中是否有所關聯
corr = df_train.corr()
cmap = sns.diverging_palette(200, 0, as_cmap = True)
sns.heatmap(
    corr,
    cmap=cmap,
    square = False,
    cbar_kws = {'shrink': .9}, #顏色條大小
    ax = plt.subplots(figsize =(12,10))[1],
    annot = True,
    annot_kws = {'fontsize':12} #字體大小
)                             

In [ ]:
#@title survive or not pie plot
#kind(圖形類型)autopct小數
plt.figure(figsize = (10,5))
df_train['Survived'].value_counts().plot(kind = 'pie', colors =['lightcoral','skyblue'], autopct = '%1.2f%%')
plt.title('Survival')
plt.ylabel('')
plt.show()

In [ ]:
#@title survived data relation
#所有生還的人其他的資料是否有相關為看不出關聯
sur = (df_train['Survived']==1)
df_sur = df_train[sur].drop(['Survived'],axis = 1)
corr = df_sur.corr()
cmap = sns.diverging_palette(200, 0, as_cmap = True)
sns.heatmap(
    corr,
    cmap=cmap,
    square = False,
    cbar_kws = {'shrink': .9}, #顏色條大小
    ax = plt.subplots(figsize =(12,10))[1],
    annot = True,
    annot_kws = {'fontsize':12} #字體大小
)                             

In [ ]:
#@title survived data plot
selected_cols = ['Sex','Pclass','Embarked','SibSp','Parch']

plt.figure( figsize =(10,len(selected_cols)*5))
gs = gridspec.GridSpec(len(selected_cols),1)
for i, col in enumerate(df_train[selected_cols]):
  ax = plt.subplot(gs[i])
  sns.countplot(df_train[col], hue = df_train.Survived, palette = ['lightcoral','skyblue'])
  ax.set_yticklabels([])
  ax.set_ylabel('Counts')
  ax.legend(loc=1)#圖例顯示位子
  for p in ax.patches:
    ax.annotate('{:,}'.format(p.get_height()), (p.get_x(),p.get_height()+1.5))
plt.show()

In [265]:
#@title Data clean def
def Data_clean_function(data):
  data = data.drop(['Name','Ticket','Cabin'],axis = 1)
  data['Sex'] = labelencoder.fit_transform(data['Sex'])
  data['Age'] = data['Age'].fillna(data['Age'].median())
  data['Embarked'] = data['Embarked'].fillna('S')
  data['Embarked'] = labelencoder.fit_transform(data['Embarked'])
  data = pd.get_dummies(data, columns = ['Embarked'])
  data['Fare'] = data['Fare'].fillna(data['Fare'].mode()[0])
  data['Family_Size'] = data['SibSp'] + data['Parch']+1
  return data

In [266]:
def Age_class(Age):
  Age['Age_class'] = np.nan
  Age.loc[Age.Age<=2,'Age_class'] = 1
  Age.loc[(Age.Age>2) & (Age.Age <=5),'Age_class'] = 2  
  Age.loc[(Age.Age>5) & (Age.Age <=11),'Age_class'] = 3
  Age.loc[(Age.Age>11) & (Age.Age <=18),'Age_class'] = 4
  Age.loc[(Age.Age>18) & (Age.Age <=30),'Age_class'] = 5
  Age.loc[(Age.Age>30) & (Age.Age <=44),'Age_class'] = 6
  Age.loc[(Age.Age>44) & (Age.Age <=60),'Age_class'] = 7
  Age.loc[Age.Age >60 ,'Age_class'] = 8    

In [267]:
df_test = Data_clean_function(df_test)
Age_class(df_test)

In [268]:
df_train = Data_clean_function(df_train)
Age_class(df_train)

In [ ]:
df_train['Family_Size'] = df_train['SibSp'] + df_train['Parch']+1
Survival_Rate = df_train[['Family_Size','Survived']].groupby('Family_Size').agg(np.mean)*100
Survival_Rate.columns = ['Survival Rate(%)']
Survival_Rate.reset_index()

In [ ]:
Survival_Rate = df_train[['Age_class','Survived',]].groupby('Age_class').agg(np.mean)*100
Survival_Rate.columns = ['Survival Rate(%)']
Survival_Rate.reset_index()

刪除看起來比較沒用的資料'Name','Ticket','Cabin'

In [52]:
df_train = df_train.drop(['Name','Ticket','Cabin'],axis = 1)

In [56]:
df_train['Sex'] = labelencoder.fit_transform(df_train['Sex'])

看是否有空值

In [167]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mode()[0])
df_train['Embarked'] = df_train['Embarked'].fillna('S')
df_train['Embarked'] = labelencoder.fit_transform(df_train['Embarked'])
df_train = pd.get_dummies(df_train, columns = ['Embarked'])

將Age用眾數填補空值，Embarked用眾數填補空值

In [269]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']

train_X = df_train[columns_X]
train_y = df_train[columns_y]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
scores

array([0.79329609, 0.78651685, 0.78651685, 0.7752809 , 0.8258427 ])

In [232]:
columns_X = set(df_train.columns) - {'Survived'}
columns_y = ['Survived']

train_X = df_train[columns_X]
train_y = df_train[columns_y]

test_X = df_test[columns_X]

In [233]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
model = linear.fit(train_X, train_y)
df_test['Survived'] = np.nan
df_test['Survived']= linear.predict(test_X)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Family_Size,Age_class,Embarked_0,Embarked_1,Embarked_2,Survived
0,892,3,1,34.5,0,0,7.8292,1,6.0,0,1,0,0.101683
1,893,3,0,47.0,1,0,7.0000,2,7.0,0,0,1,0.453996
2,894,2,1,62.0,0,0,9.6875,1,8.0,0,1,0,0.152434
3,895,3,1,27.0,0,0,8.6625,1,5.0,0,0,1,0.118296
4,896,3,0,22.0,1,1,12.2875,3,5.0,0,0,1,0.549297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,1,21.0,0,0,8.0500,1,5.0,0,0,1,0.114140
414,1306,1,0,39.0,0,0,108.9000,1,6.0,1,0,0,0.998463
415,1307,3,1,38.5,0,0,7.2500,1,6.0,0,0,1,0.062117
416,1308,3,1,21.0,0,0,8.0500,1,5.0,0,0,1,0.114181
